<a href="https://colab.research.google.com/github/batool-sajdi/project_for_prects/blob/main/LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -qU langchain-pinecone langchain-google-genai

In [12]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)


In [13]:
import time

index_name = "online-rag-project"  # change if desired

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)


In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import os

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [22]:
vector = embeddings.embed_query("i am bilding project ")

In [28]:
vector[:3]

[-0.011388907209038734, 0.011589575558900833, -0.03647366538643837]

In [29]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [30]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [31]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [34]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [33]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

# data retereve

['f5f801c2-ca62-44e2-a67e-e091ddbeb3df',
 'dc752af0-a571-42f8-bbe3-dccc4256c1f2',
 '234f3a0d-a8ee-48b3-8472-6e188b234817',
 'e6e82947-9bc5-497d-b02d-b6ef40e8d1fb',
 '198d09c6-d308-43da-8b1a-909e49bd491d',
 'cd79f679-1f65-430b-a7dc-90b3ce478734',
 'a88e3149-9600-4d43-8f1e-72cae9ed5e26',
 '8da7ceec-4839-44ad-b522-b9ade991dffa',
 '137ee5df-7bce-47dc-b675-6c7e0ab0d61c',
 'ab2ac24f-3e20-465e-8511-1739f12ee02f']

In [36]:
results = vector_store.similarity_search(
    "The top 10 soccer players in the world right now.",
    k=2,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* The top 10 soccer players in the world right now. [{'source': 'website'}]
* The top 10 soccer players in the world right now. [{'source': 'website'}]


In [35]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [37]:
results = vector_store.similarity_search_with_score(
    "Is the new iPhone worth the price? Read this review to find out.", k=1, filter={"source": "website"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.807020] Is the new iPhone worth the price? Read this review to find out. [{'source': 'website'}]


In [56]:
!pip install -qU google-generativeai

In [60]:
from langchain_google_genai import ChatGoogleGenerativeAI

def answer_to_user(query: str):
    # Vector Search
    Vector_results = vector_store.similarity_search(query, k=2)
    print(len(Vector_results))

    # Pass to Model Vector Results + User Query
    # Use generate_message instead of invoke and generate_text
    final_answer = llm.generate_message(
        prompt=f"ANSWER THIS USER QUERY:{query},HERE ARE SOME REFERNCES  TO ANWSER {Vector_results} "
    )
    # Remove the unnecessary and incorrect line
    # ChatGoogleGenerativeAI(results, query)
    return final_answer.text  # Access the text content of the response

In [61]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [62]:
def answer_to_user(query: str):
    # Vector Search
    Vector_results = vector_store.similarity_search(query, k=2)
    print(len(Vector_results))


    # Pass to Model Vector Results + User Query
    final_answer = llm.invoke(f"ANSWER THIS USER QUERY:{query},HERE ARE SOME REFERNCES  TO ANWSER {Vector_results} ")
    ChatGoogleGenerativeAI(results, query)

    return final_answer


In [65]:
def answer_to_user(query: str):
    # Vector Search
    Vector_results = vector_store.similarity_search(query, k=2)
    print(len(Vector_results))


    # Pass to Model Vector Results + User Query
    final_answer = llm.invoke(f"ANSWER THIS USER QUERY:{query},HERE ARE SOME REFERNCES  TO ANWSER {Vector_results} ")
    # Remove this line causing error, it's not needed
    # ChatGoogleGenerativeAI(results, query)

    return final_answer

In [72]:
 answer = answer_to_user("what is langchain ?")

2


In [73]:
 answer.content

"Based on the provided text, LangChain is a framework used for building projects.  The tweets suggest it's something people are using to create exciting new projects, but no further details about its functionality are given.  More information would be needed to provide a complete definition."